In [ ]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from webdriver_manager.chrome import ChromeDriverManager
# import mysql.connector
# import re
# import time

# # MySQL 연결 설정
# connection = mysql.connector.connect(
#     host='localhost',
#     user='ohgiraffers',
#     password='ohgiraffers',
#     database='test_db1',
#     charset='utf8mb4'
# )

# # Selenium 웹 드라이버 설정
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# base_url = 'https://www.bysuco.com/product?num=60&page='
# page_number = 1  # 페이지 번호 시작

# while True:
#     # 각 페이지에 접근
#     driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
#     time.sleep(2)

#     # 상품 목록에서 상세 페이지 링크 추출
#     product_links = []
#     elements = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/product/show"]')
    
#     # 상품이 없으면 루프 종료
#     if not elements:
#         print("더 이상 상품이 없습니다. 수집을 종료합니다.")
#         break

#     for element in elements:
#         href = element.get_attribute('href')
#         product_links.append(href)

#     # 각 상품 상세 페이지에서 텍스트 및 이미지 URL 추출
#     for link in product_links:
#         driver.get(link)
#         time.sleep(2)

#         try:
#             # 페이지가 완전히 로드될 때까지 기다림
#             WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap')))

#             # 필요한 데이터 추출
#             text_wrap = driver.find_element(By.CLASS_NAME, 'textWrap').text
#             name = driver.find_element(By.CLASS_NAME, 'desc.ellipsisTwo').text
#             brand_name = driver.find_element(By.CLASS_NAME, 'tit').text
#             desc_wrap = driver.find_element(By.CLASS_NAME, 'descWrap').text
            
#             # 이미지 처리 개선
#             image_urls = []
            
#             # swiper-wrapper 내 load 클래스의 img src 속성 추출
#             swiper_wrapper = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CLASS_NAME, 'swiper-slide-active'))
#             )
#             load_images = swiper_wrapper.find_elements(By.CLASS_NAME, 'load')

#             # load 클래스의 img 요소 중 src 속성을 추출
#             for img in load_images:
#                 image_url = img.get_attribute('src')
#                 print(f"이미지 URL: {image_url}")

#             # 텍스트에서 각 항목 추출
#             grade = re.search(r"\[부향률\]\s*-\s*([^\n]+)", desc_wrap)
#             grade = grade.group(1).strip() if grade else "Unknown"
            
#             top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
#             top_note = top_note.group(1).strip() if top_note else "Unknown"

#             middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
#             middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

#             base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
#             base_note = base_note.group(1).strip() if base_note else "Unknown"
            
#             single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
#             single_note = single_note.group(1).strip() if single_note else "Unknown"

#             description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
#             description = description.group(1).strip() if description else "Unknown"

#             # MySQL 데이터 삽입
#             with connection.cursor() as cursor:
#                 # 테이블 생성
#                 cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     name VARCHAR(50),
#                     brand VARCHAR(50),
#                     grade VARCHAR(50),
#                     description TEXT
#                 )''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS perfume_image (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     img_url TEXT,
#                     perfume_id BIGINT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS top_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     top_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS middle_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     middle_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS base_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     base_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')
                
#                 cursor.execute('''
#                 CREATE TABLE IF NOT EXISTS single_note (
#                     id BIGINT AUTO_INCREMENT PRIMARY KEY,
#                     perfume_id BIGINT,
#                     single_note TEXT,
#                     FOREIGN KEY (perfume_id) REFERENCES perfume(id)
#                     ON DELETE CASCADE
#                     ON UPDATE CASCADE
#                 )
#                 ''')

#                 try:
#                     # perfume 테이블에 데이터 삽입
#                     cursor.execute("""
#                         INSERT INTO perfume (name, brand, description, grade) 
#                         VALUES (%s, %s, %s, %s)
#                     """, (name, brand_name, description, grade))
#                     perfume_id = cursor.lastrowid
                    
#                     # 이미지 URL을 image 테이블에 삽입
#                     cursor.execute("INSERT INTO perfume_image (img_url, perfume_id) VALUES (%s,%s)", (image_url, perfume_id))
                    
#                     # 각 노트 삽입
#                     if top_note != "Unknown":
#                         cursor.execute("INSERT INTO top_note (perfume_id, top_note) VALUES (%s, %s)", 
#                                     (perfume_id, top_note))
#                     if middle_note != "Unknown":
#                         cursor.execute("INSERT INTO middle_note (perfume_id, middle_note) VALUES (%s, %s)", 
#                                     (perfume_id, middle_note))
#                     if base_note != "Unknown":
#                         cursor.execute("INSERT INTO base_note (perfume_id, base_note) VALUES (%s, %s)", 
#                                     (perfume_id, base_note))
#                     if single_note != "Unknown":
#                         cursor.execute("INSERT INTO single_note (perfume_id, single_note) VALUES (%s, %s)", 
#                                     (perfume_id, single_note))
                    
#                     # 모든 쿼리가 성공적으로 실행되면 커밋
#                     connection.commit()

#                 except Exception as e:
#                     # 오류 발생 시 롤백
#                     connection.rollback()
#                     print(f"데이터 삽입 중 오류 발생: {e}")
#                     continue

#         except Exception as e:
#             print(f"상세 페이지에서 정보를 추출할 수 없습니다. 오류: {e}")
#             continue  # 에러가 발생하면 다음 상품으로 넘어감

#     # 다음 페이지로 이동
#     page_number += 1

# # 드라이버 종료 및 데이터베이스 연결 종료
# driver.quit()
# connection.close()

# print("모든 데이터가 성공적으로 저장되었습니다!")

In [32]:
# 향수 디비로 만들기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import mysql.connector
import re
import time

# MySQL 연결 설정
connection = mysql.connector.connect(
    host='localhost',
    user='banghyang',
    password='banghyang',
    database='testdb3',
    charset='utf8mb4'
)

# Selenium 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)  # 명시적 대기 시간 설정
base_url = 'https://www.bysuco.com/product?num=60&page='
page_number = 1

while True:
    try:
        # 각 페이지에 접근
        driver.get(f'{base_url}{page_number}&orderBy=popular&category_id%5B%5D=2&keyword=&kind=bt')
        time.sleep(2)  # 페이지 로딩 대기 시간 증가

       # 상품 목록에서 상세 페이지 링크 추출
        product_links = []
        elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href^="/product/show"]')))
        
        if not elements:
            print("더 이상 상품이 없습니다. 수집을 종료합니다.")
            break

        # 링크 수집
        for element in elements:
            href = element.get_attribute('href')
            if href not in product_links:
                product_links.append(href)

        # 각 상품 상세 페이지 처리
        for link in product_links:
            try:
                driver.get(link)
                time.sleep(2)  # 페이지 로딩 대기

                # 기본 정보 추출
                try:
                    name_kr = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'desc.ellipsisTwo'))).text
                    
                    # 제외할 키워드 체크
                    if any(keyword in name_kr for keyword in ['벌크제품', '세트', '리필', '선물패키지', '스프레이', '오일']):
                        print(f"제외된 상품: {name_kr}")
                        continue  # 다음 상품으로 건너뜀
                        
                    print(f"상품명: {name_kr}")  # 디버깅용 출력
                    
                    name_en = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'desc.enDesc.ellipsisTwo'))).text
                    print(f"상품명: {name_en}")  # 디버깅용 출력
                    
                    brand_name = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'tit'))).text
                    print(f"브랜드: {brand_name}")  # 디버깅용 출력
                    
                    desc_wrap = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'descWrap'))).text
                    print("설명 텍스트 추출 완료")  # 디버깅용 출력
                    
                    ingredients_description = ""
                    
                    try:
                        notice_tables = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'noticeTable')))
    
                        if len(notice_tables) > 1:
                            second_notice_table = notice_tables[1]  # 두 번째 테이블 선택

                            rows = second_notice_table.find_elements(By.TAG_NAME, 'tr')

                            for row in rows:
                                th_elements = row.find_elements(By.TAG_NAME, 'th')
                                
                                # 각 <th>에 대해 "표시성분"을 확인
                                for th_element in th_elements:
                                    if th_element.text.strip() == "표시성분":
                                        # 해당 <th>에 맞는 <td>에서 ingredients description 추출
                                        td_element = row.find_element(By.TAG_NAME, 'td')
                                        ingredients_description = td_element.text.strip()
                                        print(f"표시성분 설명: {ingredients_description}")
                                        break
                    except TimeoutException as e:
                        print(f"테이블 로딩 중 오류 발생: {e}")
                    except Exception as e:
                        print(f"기타 오류 발생: {e}")
                    
                    # 사이즈 옵션 추출 부분 수정
                    try:    
                        print("사이즈 옵션 추출 시작")
                        # 명시적 대기 추가
                        time.sleep(1)
                        
                        # 모든 optionBtn 찾기 (상태와 관계없이)
                        option_buttons = driver.find_elements(By.CSS_SELECTOR, 'button.optionBtn')
                        print(f"찾은 버튼 수: {len(option_buttons)}")
                        
                        size_options = []
                        for button in option_buttons:
                            try:
                                # i 태그의 ellipsisTwo 클래스를 가진 요소 찾기
                                size_text = button.find_element(By.CSS_SELECTOR, 'i.ellipsisTwo').text.strip()
                                if size_text:
                                    size_options.append(size_text)
                                    print(f"추출된 옵션: {size_text}")
                            except Exception as e:
                                print(f"버튼 처리 중 에러: {str(e)}")
                                continue
                        
                        size_options_str = ','.join(size_options) if size_options else 'No size options'
                        print(f"최종 사이즈 옵션: {size_options_str}")
                        
                    except Exception as e:
                        print(f"사이즈 옵션 추출 중 에러: {str(e)}")
                        size_options_str = 'No size options'
                    
                except TimeoutException as e:
                    print(f"기본 정보 추출 중 타임아웃: {e}")
                    continue

                # 이미지 처리
                image_urls = []
                try:
                    # 모든 이미지를 포함하는 컨테이너 찾기
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'swiperWrap')))
                    
                    # 이미지 슬라이더의 모든 이미지 요소 찾기
                    image_elements = driver.find_elements(By.CSS_SELECTOR, '.swiperWrap img')
                    
                    # 각 이미지의 src 속성 가져오기
                    for img in image_elements:
                        src = img.get_attribute('src')
                        if src and src not in image_urls:
                            # webp2w-600 형식의 URL을 원본 URL로 변환
                            original_url = src.replace('webp2w-600', 'webp')
                            image_urls.append(original_url)
                            print(f"이미지 URL 추가됨: {original_url}")

                except Exception as e:
                    print(f"이미지 처리 중 오류 발생: {e}")

                print(f"찾은 총 이미지 URL 수: {len(image_urls)}")

                # 텍스트 정보 추출
                grade = re.search(r"\[부향률\]\s*-\s*([^\n]+)", desc_wrap)
                grade = grade.group(1).strip() if grade else "Unknown"
                
                top_note = re.search(r"\[메인 노트\]\s*-\s*탑 노트:([^\n]+)", desc_wrap)
                top_note = top_note.group(1).strip() if top_note else "Unknown"

                middle_note = re.search(r"미들 노트:([^\n]+)", desc_wrap)
                middle_note = middle_note.group(1).strip() if middle_note else "Unknown"

                base_note = re.search(r"베이스 노트:([^\n]+)", desc_wrap)
                base_note = base_note.group(1).strip() if base_note else "Unknown"
                
                single_note = re.search(r"싱글 노트:([^\n]+)", desc_wrap)
                single_note = single_note.group(1).strip() if single_note else "Unknown"

                description = re.search(r"\[향 설명\]\s*-\s*([^\n]+)", desc_wrap)
                description = description.group(1).strip() if description else "Unknown"
                
                main_accord = re.search(r"\[메인 어코드\]\s*-\s*([^\n]+)", desc_wrap)
                main_accord = main_accord.group(1).strip() if main_accord else "Unknown"

                # MySQL 데이터 삽입
                with connection.cursor() as cursor:
                    # 테이블 생성
                    cursor.execute('''CREATE TABLE IF NOT EXISTS perfume (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        name_kr VARCHAR(50),
                        name_en VARCHAR(50),
                        brand VARCHAR(50),
                        grade VARCHAR(50),
                        main_accord VARCHAR(50),
                        size_options TEXT,
                        description TEXT,
                        ingredients TEXT
                    )''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS perfume_image (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        img_url TEXT,
                        perfume_id BIGINT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS top_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        top_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS middle_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        middle_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS base_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        base_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')
                    
                    cursor.execute('''
                    CREATE TABLE IF NOT EXISTS single_note (
                        id BIGINT AUTO_INCREMENT PRIMARY KEY,
                        perfume_id BIGINT,
                        single_note TEXT,
                        FOREIGN KEY (perfume_id) REFERENCES perfume(id)
                        ON DELETE CASCADE
                        ON UPDATE CASCADE
                    )
                    ''')

                    try:
                        # perfume 테이블에 데이터 삽입
                        cursor.execute("""
                            INSERT INTO perfume (name_kr, name_en, brand, main_accord, size_options, description, grade, ingredients) 
                            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                        """, (name_kr, name_en, brand_name, main_accord, size_options_str, description, grade, ingredients_description))
                        perfume_id = cursor.lastrowid
                        
                        # 이미지 URL 저장
                        for url in image_urls:
                            cursor.execute("""
                                INSERT INTO perfume_image (img_url, perfume_id) 
                                VALUES (%s, %s)
                            """, (url, perfume_id))
                        
                        # 노트 정보 저장
                        if top_note != "Unknown":
                            cursor.execute("INSERT INTO top_note (perfume_id, top_note) VALUES (%s, %s)", 
                                        (perfume_id, top_note))
                        if middle_note != "Unknown":
                            cursor.execute("INSERT INTO middle_note (perfume_id, middle_note) VALUES (%s, %s)", 
                                        (perfume_id, middle_note))
                        if base_note != "Unknown":
                            cursor.execute("INSERT INTO base_note (perfume_id, base_note) VALUES (%s, %s)", 
                                        (perfume_id, base_note))
                        if single_note != "Unknown":
                            cursor.execute("INSERT INTO single_note (perfume_id, single_note) VALUES (%s, %s)", 
                                        (perfume_id, single_note))
                        
                        connection.commit()
                        print(f"상품 '{name_kr}' 데이터 저장 완료")

                    except Exception as e:
                        connection.rollback()
                        print(f"데이터베이스 저장 중 오류: {e}")
                        continue

            except Exception as e:
                print(f"상품 페이지 처리 중 오류 발생: {e}")
                continue

        page_number += 1

    except Exception as e:
        print(f"페이지 처리 중 오류 발생: {e}")
        break

driver.quit()
connection.close()
print("크롤링이 완료되었습니다!")

상품명: 플레르 드 뽀 오 드 퍼퓸
상품명: FLEUR DE PEAU EDP
브랜드: 딥티크
설명 텍스트 추출 완료
표시성분 설명: 변성알코올, 향료, 정제수, 에칠헥실메톡시신나메이트, 에칠헥실살리실레이트, 부틸메톡시디벤조일메탄, 비에이치티, 리모넨, 리날룰, 알파-아이소메틸아이오논, 파네솔, 제라니올, 시트랄, 시트로넬올
사이즈 옵션 추출 시작
찾은 버튼 수: 2
추출된 옵션: 75ml
최종 사이즈 옵션: 75ml
이미지 URL 추가됨: https://cf.bysuco.net/674f1342fa588659762bd7e538e87297_cvt.webp?w=600
찾은 총 이미지 URL 수: 1
상품 '플레르 드 뽀 오 드 퍼퓸' 데이터 저장 완료
상품명: 어나더 13 오 드 퍼퓸
상품명: ANOTHER 13 EDP
브랜드: 르 라보
설명 텍스트 추출 완료
표시성분 설명: 변성알코올,향료,정제수,벤질살리실레이트,벤질벤조에이트,비에이치티 [ILN48648]
제공된 성분은 동일 제품이라도 경우에 따라 변경될 수 있습니다.
사이즈 옵션 추출 시작
찾은 버튼 수: 4
추출된 옵션: 50ml
추출된 옵션: 100ml
최종 사이즈 옵션: 50ml,100ml
이미지 URL 추가됨: https://cf.bysuco.net/ea50f67fafb51e48a8e09764e9635e4b.jpg?w=600
이미지 URL 추가됨: https://cf.bysuco.net/46f40b17478c7c08a9fcc331df231123.jpg?w=600
찾은 총 이미지 URL 수: 2
상품 '어나더 13 오 드 퍼퓸' 데이터 저장 완료
상품명: 오르페옹 오 드 퍼퓸
상품명: ORPHEON EDP
브랜드: 딥티크
설명 텍스트 추출 완료
표시성분 설명: 변성알코올, 향료, 정제수, 벤조익애씨드, 메틸벤조에이트, 벤질살리실레이트, 리모넨, 리날룰, 알파-아이소메틸아이오논, 하이드록시시트로넬알, 제라니올, 시트로넬올, 쿠마린, 시트랄, 벤질벤조에이트, 벤질알코올, 신남알, 헥실신남알, 아이소유제놀
사이

In [9]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import Error
from urllib.parse import urljoin

try:
    connection = mysql.connector.connect(
        host='localhost',
        user='banghyang',
        password='banghyang',
        database='testdb2',
        charset='utf8mb4',
        use_unicode=True
    )
    
    cursor = connection.cursor(buffered=True)

    # 테이블 생성
    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS line (
            id BIGINT PRIMARY KEY AUTO_INCREMENT,
            name VARCHAR(255),
            content VARCHAR(255)
        )
    ''')

    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS spice (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            name_en VARCHAR(255) NOT NULL,
            name_kr VARCHAR(255) NOT NULL,
            content_en TEXT,
            content_kr TEXT,
            line_id BIGINT,
            FOREIGN KEY (line_id) REFERENCES line(id) ON DELETE CASCADE
        )
    ''')

    cursor.execute(''' 
        CREATE TABLE IF NOT EXISTS spice_image (
            id BIGINT AUTO_INCREMENT PRIMARY KEY,
            img_url TEXT NOT NULL,
            spice_id BIGINT,
            FOREIGN KEY (spice_id) REFERENCES spice(id) ON DELETE CASCADE
        )
    ''')

    # line 데이터 삽입
    cursor.execute(''' 
        INSERT IGNORE INTO line (name, content) VALUES
            ( 'Spicy', '후추나 시나몬 같은 향신료에서 느껴지는 따뜻하고 자극적인 향. 강렬하면서도 대담한 느낌으로, 어두운 조명 아래 와인 한 잔을 즐기는 우아한 저녁 모임에 어울립니다. 따뜻한 니트나 가죽 재킷과도 잘 어울리는 계열입니다.'),
            ('Fruity', '잘 익은 과일의 달콤함과 상큼함이 조화를 이루는 향. 햇살이 비치는 공원에서 피크닉을 즐기거나, 친구들과 가벼운 브런치를 즐길 때 어울립니다. 발랄하고 사랑스러운 이미지를 연출하는 데 적합합니다.'),
            ('Citrus', '갓 짜낸 오렌지 주스처럼 톡 쏘는 상쾌함과 신선함을 주는 향. 무더운 여름날 시원한 레모네이드를 마시는 순간이나, 활기찬 아침 출근길에 어울립니다. 에너지를 북돋아 주는 활력의 계열입니다.'),
            ('Green', '비 온 뒤 숲속의 맑은 공기나, 잘 정돈된 잔디밭 위의 신선함을 담은 향. 여유로운 산책이나 차분한 독서 시간과 잘 어울리며, 자연 속에 있는 듯한 평온함을 줍니다.'),
            ('Aldehyde', '고급스러운 비누향과 깨끗함이 돋보이는 향. 깔끔한 화이트 셔츠나 포멀한 정장 차림에 잘 어울리며, 모던하고 세련된 이미지를 강조합니다. 회의실에서의 자신감 넘치는 프레젠테이션이나, 갤러리 방문 같은 정적인 분위기에 적합합니다.'),
            ('Aquatic', '시원한 바닷바람과 맑은 물방울을 연상시키는 청량한 향. 해변에서 여름 바캉스를 즐기거나, 수영장에서 느껴지는 잔잔한 물결의 고요함을 담았습니다. 더운 날의 피로를 씻어내는 상쾌함을 제공합니다.'),
            ('Fougere', '부드럽고 편안한 느낌을 주는 라벤더와 오크모스의 조화로운 향. 클래식한 이탈리안 정원에서 느껴지는 고급스러운 감성과 여유로움을 떠올리게 합니다. 남성적인 매력을 부드럽게 표현할 때 적합합니다.'),
            ('Gourmand', '따뜻한 초콜릿 케이크나 갓 구운 쿠키처럼 달콤하고 유혹적인 향. 디저트 카페에서 느낄 수 있는 포근한 향기로, 사랑스럽고 다정한 이미지를 연출합니다. 겨울 저녁에 특히 잘 어울리는 계열입니다.'),
            ('Woody', '나무의 따뜻함과 자연스러운 우아함이 느껴지는 향. 모닥불 옆에서 느껴지는 평온함과, 빈티지 가구에서 풍기는 고급스러운 분위기를 연상시킵니다. 클래식한 스타일을 선호하는 이들에게 추천합니다.'),
            ('Oriental', '달콤하고 부드러운 바닐라와 고혹적인 앰버가 어우러져 센슈얼한 매력을 발산하는 향. 붉은 실크 드레스나 황혼의 무드에 어울리며, 이국적이고 매혹적인 분위기를 만들어냅니다.'),
            ('Floral', '화사한 꽃다발처럼 우아하고 여성스러운 향. 봄날 벚꽃 아래를 걷는 듯한 로맨틱한 분위기를 만들어주며, 부드러운 드레스 차림에 잘 어울립니다.'),
            ('Musk', '따뜻하면서도 부드럽게 이성을 자극하는 매혹적인 향. 깨끗하고 포근한 머스크의 베이스는, 자연스러운 아름다움을 강조하는 분위기를 연출합니다. 특별한 데이트나 은밀한 저녁 시간에 적합합니다.'),
            ('Powdery', '포근한 담요처럼 감싸주는 부드럽고 따뜻한 향. 차분하고 편안한 분위기를 연출하며, 휴식이 필요한 공간이나 밤 시간에 잘 어울립니다.'),
            ('Tobacco Leather', '스모키한 담배와 묵직한 가죽 향이 어우러져 강렬한 남성미를 느낄 수 있는 향. 한겨울 가죽 재킷을 입고 있는 도시 남성의 세련된 매력을 연상시킵니다. 중성적이면서도 고급스러운 분위기를 연출합니다.')
            ''')

    url = "https://www.fragrantica.com/notes/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    notes = soup.find_all('div', class_='cell small-6 medium-4 large-3 text-center notebox')

    if not notes:
        print("향료 데이터를 찾지 못했습니다.")
    else:
        for note in notes:
            try:
                a_tag = note.find('a')
                if not a_tag:
                    continue
                
                name_en = a_tag.text.strip()
                note_url = urljoin("https://www.fragrantica.com", a_tag.get('href'))
                img_tag = note.find('img')
                image_url = img_tag.get('src') if img_tag else None
                
                # 향료 상세 페이지로 요청
                detail_response = requests.get(note_url, headers=headers)
                detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

                # content_en 추출 부분 수정
                content_en = ''  # 기본값을 빈 문자열로 설정
                callout_tag = detail_soup.find('div', class_='cell callout')
                if callout_tag:
                    p_tag = callout_tag.find('p')
                    if p_tag:
                        content_en = p_tag.get_text(strip=True)
                        # "Odor Profile:" 부분 제거
                        if content_en.startswith("Odor Profile:"):
                            content_en = content_en[13:].strip()  # "Odor Profile:" 이후 텍스트만 가져오기
                        elif content_en.lower().startswith("odor profile:"):
                            content_en = content_en[13:].strip()

                content_kr = ''  # content_kr은 추후 추가
                name_kr = ''  # name_kr은 추후 추가
                
                # spice 데이터 삽입 (content가 없어도 빈 문자열로 저장)
                cursor.execute(
                    "INSERT INTO spice (name_en, name_kr, content_en, content_kr, line_id) VALUES (%s, %s, %s, %s, %s)",
                    (name_en, name_kr, content_en, content_kr, 1)
                )
                spice_id = cursor.lastrowid

                # 이미지 데이터 삽입
                if image_url:
                    cursor.execute("INSERT INTO spice_image (img_url, spice_id) VALUES (%s, %s)", (image_url, spice_id))
                
                print(f"데이터 삽입 성공 - Name: {name_en}, Content: {content_en}, Image URL: {image_url}")
                connection.commit()

            except Exception as e:
                print(f"항목 처리 중 오류 발생: {e}")
                connection.rollback()
                continue

        print("모든 데이터가 성공적으로 저장되었습니다!")

except Exception as e:
    print(f"프로그램 실행 중 오류 발생: {e}")
    if connection:
        connection.rollback()

finally:
    if connection and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL 연결이 종료되었습니다.")

데이터 삽입 성공 - Name: Bergamot, Content: citrusy, bitter & tart, elegant, light note with mild spicy tone, complex with nuances of fruit and aromatic elements, reminiscent of eau de Cologne, it flavors Earl Grey tea., Image URL: https://fimgs.net/mdimg/sastojci/m.75.jpg?1733161266
데이터 삽입 성공 - Name: Bigarade, Content: , Image URL: https://fimgs.net/mdimg/sastojci/m.1083.jpg?1733161258
데이터 삽입 성공 - Name: Bitter Orange, Content: It is inedible tart citrus, known as "bigarade". It is intensely scented., Image URL: https://fimgs.net/mdimg/sastojci/m.79.jpg?1733161266
데이터 삽입 성공 - Name: Blood Orange, Content: It is a mutation of the sweet orange, very sweet and juicy, with a sweet taste and smell., Image URL: https://fimgs.net/mdimg/sastojci/m.286.jpg?1733161262
데이터 삽입 성공 - Name: Buddha's hand, Content: citrus, lemon, fresh, sweet, green, woody and herbal facetes, Image URL: https://fimgs.net/mdimg/sastojci/m.1589.jpg?1733161259
데이터 삽입 성공 - Name: Calamansi, Content: Its aroma is quite acidic and r

In [ ]:
# 중복 향수 데이터 처리
import pandas as pd
import mysql.connector
from typing import Dict, List, Set, Tuple

# MySQL 연결 설정
def get_connection():
    return mysql.connector.connect(
        host='localhost',
        user='banghyang',
        password='banghyang',
        database='testdb3',
        charset='utf8mb4'
    )

def get_duplicates(perfumes_df: pd.DataFrame) -> pd.DataFrame:
    """
    중복된 제품들을 찾아서 반환하는 함수
    """
    # 제품별 그룹화
    duplicates = perfumes_df.groupby(['name_kr', 'name_en', 'brand']).agg({
        'id': list,
        'size_options': list
    }).reset_index()
    
    # 2개 이상의 항목이 있는 그룹만 필터링
    duplicates = duplicates[duplicates['id'].map(len) > 1]
    
    return duplicates

def consolidate_duplicates(dry_run: bool = True) -> Dict:
    """
    중복된 향수 데이터를 통합하는 함수
    """
    conn = get_connection()
    conn.autocommit = True  # dry run 모드에서는 자동 커밋 설정
    cursor = conn.cursor(dictionary=True)
    
    # 제품 데이터 로드
    cursor.execute("""
        SELECT id, name_kr, name_en, brand, size_options 
        FROM perfume
    """)
    perfumes_df = pd.DataFrame(cursor.fetchall())
     
    # 중복 찾기
    duplicates = get_duplicates(perfumes_df)
    
    updates = []
    deletes = []
    
    # 중복 항목들 처리
    for _, row in duplicates.iterrows():
        main_id = min(row['id'])
        other_ids = [id for id in row['id'] if id != main_id]
        
        # size_options 통합
        all_sizes = set()
        for size_opt in row['size_options']:
            sizes = [s.strip() for s in size_opt.split(',')]
            all_sizes.update(sizes)
        
        combined_sizes = ', '.join(sorted(all_sizes))
        
        updates.append({
            'id': main_id,
            'size_options': combined_sizes,
            'product_name': f"{row['name_kr']} ({row['brand']})"
        })
        deletes.extend(other_ids)
    
    # dry_run이 False일 때만 실제 변경 수행
    if not dry_run:
        try:
            conn.autocommit = False  # 실제 변경 모드에서는 자동 커밋 해제
            
            # size_options 업데이트
            for update in updates:
                cursor.execute("""
                    UPDATE perfume 
                    SET size_options = %s 
                    WHERE id = %s
                """, (update['size_options'], update['id']))
            
            # 이미지 처리 - 각 중복 그룹별로 처리
            for _, row in duplicates.iterrows():
                main_id = min(row['id'])  # 해당 그룹의 메인 ID
                other_ids = [id for id in row['id'] if id != main_id]  # 해당 그룹에서 병합될 ID들
                
                # 해당 그룹의 다른 ID들의 이미지만 메인 ID로 업데이트
                for old_id in other_ids:
                    cursor.execute("""
                        UPDATE perfume_image 
                        SET perfume_id = %s 
                        WHERE perfume_id = %s
                    """, (main_id, old_id))
            
            # 중복 제품 삭제
            for delete_id in deletes:
                cursor.execute("""
                    DELETE FROM perfume 
                    WHERE id = %s
                """, (delete_id,))
            
            # 트랜잭션 커밋
            conn.commit()
            
        except Exception as e:
            # 오류 발생시 롤백
            conn.rollback()
            print(f"Error occurred: {e}")
            raise e
        finally:
            conn.autocommit = True  # 원래대로 복구
    
    cursor.close()
    conn.close()
    
    # 결과 리포트 생성
    report = {
        'total_duplicates': len(duplicates),
        'to_be_updated': [u['product_name'] for u in updates],
        'to_be_deleted': len(deletes),
        'dry_run': dry_run
    }
    
    return report

def print_report(report: Dict):
    """결과 리포트를 보기 좋게 출력하는 함수"""
    print("\n=== 중복 데이터 처리 리포트 ===")
    print(f"총 중복 그룹 수: {report['total_duplicates']}")
    print(f"처리 모드: {'테스트(dry run)' if report['dry_run'] else '실제 적용'}")
    print(f"삭제될 제품 수: {report['to_be_deleted']}")
    
    print("\n=== 업데이트될 제품 전체 목록 ===")
    for idx, product in enumerate(sorted(report['to_be_updated']), 1):
        print(f"{idx:3d}. {product}")

# 사용 예시
if __name__ == "__main__":
    # 먼저 dry run으로 테스트
    print("테스트 실행 중...")
    test_report = consolidate_duplicates(dry_run=True)
    print_report(test_report)
    
    user_input = input("\n변경사항을 적용하시겠습니까? (yes/no): ")
    if user_input.lower() == 'yes':
        real_report = consolidate_duplicates(dry_run=False)
        print("\n변경사항이 적용되었습니다.")
        print_report(real_report)

테스트 실행 중...

=== 중복 데이터 처리 리포트 ===
총 중복 그룹 수: 159
처리 모드: 테스트(dry run)
삭제될 제품 수: 177

=== 업데이트될 제품 전체 목록 ===
  1. 001 맨 오 드 뚜왈렛 (로에베)
  2. 724 오 드 퍼퓸 (메종 프란시스 커정)
  3. H24 오 드 뚜왈렛 (에르메스)
  4. 그린 네롤리 오 드 퍼퓸 (크리드)
  5. 그린 아이리쉬 트위드 오 드 퍼퓸 (크리드)
  6. 길티 블랙 포 힘 오 드 뚜왈렛 (구찌)
  7. 길티 인텐스 뿌르 팜므 오 드 퍼퓸 (구찌)
  8. 꼴론 앙델레빌 오 드 퍼퓸 (프레데릭 말)
  9. 끌로에 시그니처 나츄렐 인텐스 오 드 퍼퓸 (끌로에)
 10. 낫 어 퍼퓸 오 드 퍼퓸 (줄리엣 헤즈 어 건)
 11. 넥타린 블로썸 앤 허니 오 드 코롱 (조 말론)
 12. 노마드 오 드 퍼퓸 (끌로에)
 13. 누아 에삐쓰 오 드 퍼퓸 (프레데릭 말)
 14. 더 나이트 오 드 퍼퓸 (프레데릭 말)
 15. 덩 떼 브하 오 드 퍼퓸 (프레데릭 말)
 16. 듀(II) 오 드 퍼퓸 (트루동)
 17. 드리스 반 노튼 오 드 퍼퓸 (프레데릭 말)
 18. 떼 누아 29 오 드 퍼퓸 (르 라보)
 19. 떼 마차 26 오 드 퍼퓸 (르 라보)
 20. 떼르 데르메스 오 드 뚜왈렛 (에르메스)
 21. 라임 바질 앤 만다린 오 드 코롱 (조 말론)
 22. 랄리벨라 오 드 퍼퓸 (메모)
 23. 러브 인 화이트 썸머 오 드 퍼퓸 (크리드)
 24. 레더 오 드 퍼퓸 (멜린앤게츠)
 25. 레드 로즈 오 드 코롱 (조 말론)
 26. 로 데 헤스프리데스 오 드 뚜왈렛 (딥티크)
 27. 로 디베 오 드 뚜왈렛 (프레데릭 말)
 28. 로 아 라 로즈 오 드 뚜왈렛 (메종 프란시스 커정)
 29. 로사 노빌레 오 드 퍼퓸 (아쿠아 디 파르마)
 30. 로얄 프린세스 우드 오 드 퍼퓸 (크리드)
 31. 로즈 31 오 드 퍼퓸 (르 라보)
 32. 로즈 앤 뀌흐 오 드 퍼퓸 (프레데